# LR parser
LR parser is a bottom-up parser that can parse context-free languages in linear time,
i.e. it reads input tokens, contatenates them into AST nodes in hope to build tree at the end.
This notebook contains an implementation of LR(0) parser according to the Dragon Book.